##ModelFactory

Create models for easy lookup.

FUTURE: use metaclass programming.

In [30]:
debug = True

DRIVE_PATH = "/content/drive/MyDrive/data606"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "/src/"

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
%cd $SCRIPT_PATH

/content/drive/MyDrive/data606/src


In [33]:
# Load model classes
%run -i "./Model_Densev1.ipynb"
%run -i "./Model_LSTMv2.ipynb"
%run -i "./Model_LSTMv3.ipynb"

In [34]:

class ModelFactory():
  """
  Construct a ModelFactory that provides keyword access to models for training.
  """
  def __init__(self, window_size=30, num_labels=1, num_epochs=300, debug=False):

    self.debug = debug

    self.window_size = window_size
    self.num_labels = num_labels
    self.num_epochs = num_epochs

    self.init_models()


  def __repr__(self):
    """
    Print object stats.
    """
    return '\n'.join([
         f'window_size: {self.window_size}',
         f'num_labels: {self.num_labels}',
         f'num_epochs: {self.num_epochs}'
        ])

  def init_models(self):
    """
    FUTURE: discover models
    """
    self.models = {}

    #TODO please use something dynamic
    model = Model_Densev1(window_size=self.window_size,
                          num_labels=self.num_labels,
                          num_epochs=self.num_epochs, debug=self.debug)

    self.models[model.get_name()] = model

    model = Model_LSTMv2(window_size=self.window_size,
                          num_labels=self.num_labels,
                          num_epochs=self.num_epochs, debug=self.debug)

    self.models[model.get_name()] = model

    model = Model_LSTMv3(window_size=self.window_size,
                          num_labels=self.num_labels,
                          num_epochs=self.num_epochs, debug=self.debug)

    self.models[model.get_name()] = model

  def get(self, model_name):
    return self.models[model_name]


---

**Unit testing**

---

In [35]:
WG_UNIT_TEST = True

In [36]:
if WG_UNIT_TEST:

  print(f'-------Case 1: get single model -----------')
  mf = ModelFactory()

  model = mf.get("Densev1")
  assert(model)
  print(model)

-------Case 1: get single model -----------


In [37]:
if WG_UNIT_TEST:

  print(f'-------Case 2: get all models -----------')
  mf = ModelFactory()

  models = ["Densev1",'LSTMv2','LSTMv3']

  for m in models:
    model = mf.get(m)
    assert(model)
    print(model)

-------Case 2: get all models -----------
